# Pull all features for an area

In this notebook, I'm developing and testing code that pulls and standardizes all 
of the map features that we are interested in, for a box of a given size.

In [ ]:
import numpy as np
import pandas as pd
import pyproj
import shapely
import osmnx

import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter

from geo_encodings import draw_shape

import polars as pl
pl.Config.set_tbl_rows(50)


## Parameters

In [ ]:
# AOIs are squares of this dimension, in meters.
aoi_size = 1000

# For testing
lat, lon = 42.981163, -70.946524 # Exeter NH


## Preliminaries

In [ ]:
# Define a local map projection

center_lat = lat
center_lon = lon

proj_def = f"""
+proj=tmerc +lat_0={center_lat} +lon_0={center_lon} 
+k=1.0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs
"""

ltm_crs = pyproj.CRS.from_proj4(proj_def)
wgs84_crs = pyproj.CRS.from_epsg(4326)
proj_forward = pyproj.Transformer.from_crs(wgs84_crs, ltm_crs, always_xy=True).transform
proj_inverse = pyproj.Transformer.from_crs(ltm_crs, wgs84_crs, always_xy=True).transform



In [ ]:
# Use that projection to define lon/lat nbounds. Make sure the bounds go 
# a little farther out than necessary to avoid edge artifacts from map projections.
dd = aoi_size * 0.6

lon0, lat0 = proj_inverse(-dd, -dd)
lon1, lat1 = proj_inverse(+dd, +dd)
print(lon0, lat0, lon1, lat1)

In [ ]:
# Define the boundng box to be used to query OSM.
query_bounds = [lon0, lat0, lon1, lat1]
center_lon = (lon0 + lon1) / 2
center_lat = (lat0 + lat1) / 2

In [ ]:
tags = {
    'landuse': True,
    'place': True,
    'highway': True,
    'railway': True,
    'aeroway': True,
    'bridge': True,
    'tunnel': True,
    'power': True,
    'natural': True,
    'waterway': True,
    'landcover': True,
    'building': True,
    'amenity': True,
    'shop': True,
    'leisure': True
}
# features = osmnx.features.features_from_bbox(query_bounds, tags=tags).reset_index()
# print('%d features' % len(features))

In [ ]:
elements = []

for key in ['highway']:
    features = osmnx.features.features_from_bbox(query_bounds, tags=tags).reset_index()
    for rec in features.to_dict('records'):
        geomxy = shapely.ops.transform(proj_forward, rec['geometry'])
        gtype = geomxy.geom_type
    
    


In [ ]:
features.iloc[0].to_dict()